In [1]:
import numpy as np
from scipy.linalg import block_diag
from matrices import *

In [2]:
#debugging the VFH matrix

In [4]:
A=-VFH_function(3)

In [3]:
def VFH_function(K):
    '''
    input: K: the number of fractal iterations we want to use to define our matrix 
    
    output: the VFH matrix of dimension 5**k x 5**k
    '''

    H=np.diag(-2*np.ones(5))
    H[0]=np.ones(5)
    H[:,0]=np.ones(5)
    H[0,0]=-4

    p_11=3
    p_21=2
    p_31=5
    p_41=4

    p_1k_1=p_11
    p_2k_1=p_21
    p_3k_1=p_31
    p_4k_1=p_41

    if K<=0:
        raise ValueError('k has to be strictly positive')
    
    for k in range(2,K+1):
        #we define H_i depending on H_{i-1}
        #we already defined H_1 before the loop
        size=H.shape[0]
    
        if k>2:
            p_1k=5**(k-2)*(p_11-1)+p_1k_1
            p_2k=5**(k-2)*(p_21-1)+p_2k_1
            p_3k=5**(k-2)*(p_31-1)+p_3k_1
            p_4k=5**(k-2)*(p_41-1)+p_4k_1
        else:
            p_1k=p_11
            p_2k=p_21
            p_3k=p_31
            p_4k=p_41
    
        #putting -1 in the index to compensate for the indexing going from 0 to size-1 in numpy
        V_1=np.zeros((size, size))
        V_1[p_1k-1,p_2k-1]=1
        V_2=np.zeros((size, size))
        V_2[p_2k-1, p_1k-1]=1
        V_3=np.zeros((size, size))
        V_3[p_3k-1, p_4k-1]=1
        V_4=np.zeros((size, size))
        V_4[p_4k-1, p_3k-1]=1
    
        p_1k_1=p_1k
        p_2k_1=p_2k
        p_3k_1=p_3k
        p_4k_1=p_4k
    
        #Since we have a lot of block matrices, easier to explicitely complete it
        H_k=block_diag(*([H] * 5))
        H_k[size:2*size, 0:size]=V_1
        H_k[2*size:3*size, 0:size]=V_2
        H_k[3*size:4*size, 0:size]=V_3
        H_k[4*size:5*size, 0:size]=V_4
        H_k[0:size, size:2*size]=V_1.T
        H_k[0:size, 2*size:3*size]=V_2.T
        H_k[0:size, 3*size:4*size]=V_3.T
        H_k[0:size, 4*size:5*size]=V_4.T
    
        H=H_k
    return H

In [5]:
A=-VFH_function(4)

In [6]:
if not (A==A.T).all():
    print("A is not symmetric. Please choose A such that A=A.T")
    print("A =",A)
if (np.linalg.eigvals(A)<=0).any():
    print("The matrix A should be positive definite.")
    print("eigenvalues of A:", np.linalg.eigvals(A))

In [16]:
#!!! VFH isn't positive definite but -VFH is so I think we should just work with -VFH

### The code from Matlab on how to plot the Wathen matrix (saved as Markdown)! 

wathen_ge() returns the Wathen matrix, using general (GE) storage.
%
%  Discussion:
%
%    The Wathen matrix is a finite element matrix which is sparse.
%
%    The entries of the matrix depend in part on a physical quantity
%    related to density.  That density is here assigned random values between
%    0 and 100.
%
%    The matrix order N is determined by the input quantities NX and NY,
%    which would usually be the number of elements in the X and Y directions.
%    The value of N is
%
%      N = 3*NX*NY + 2*NX + 2*NY + 1,
%
%    The matrix is the consistent mass matrix for a regular NX by NY grid
%    of 8 node serendipity elements.
%
%    The local element numbering is
%
%      3--2--1
%      |     |
%      4     8
%      |     |
%      5--6--7
%
%    Here is an illustration for NX = 3, NY = 2:
%
%     23-24-25-26-27-28-29
%      |     |     |     |
%     19    20    21    22
%      |     |     |     |
%     12-13-14-15-16-17-18
%      |     |     |     |
%      8     9    10    11
%      |     |     |     |
%      1--2--3--4--5--6--7
%
%    For this example, the total number of nodes is, as expected,
%
%      N = 3 * 3 * 2 + 2 * 2 + 2 * 3 + 1 = 29
%
%    The matrix is symmetric positive definite for any positive values of the
%    density RHO(X,Y).
%
%  Licensing:
%
%    This code is distributed under the GNU LGPL license.
%
%  Modified:
%
%    07 July 2014
%
%  Author:
%
%    John Burkardt
%
%  Reference:
%
%    Nicholas Higham,
%    Algorithm 694: A Collection of Test Matrices in MATLAB,
%    ACM Transactions on Mathematical Software,
%    Volume 17, Number 3, September 1991, pages 289-305.
%
%    Andrew Wathen,
%    Realistic eigenvalue bounds for the Galerkin mass matrix,
%    IMA Journal of Numerical Analysis,
%    Volume 7, Number 4, October 1987, pages 449-457.
%
%  Input:
%
%    integer NX, NY, values which determine the size of the matrix.
%
%    integer N, the number of variables.
%
%  Output:
%
%    real A(N,N), the matrix.
%
  if ( nargin < 1 )
    fprintf ( 1, '\n' );
    fprintf ( 1, 'wathen_ge(): Fatal error!\n' );
    fprintf ( 1, '  Not enough input.\n' );
    error ( 'wathen_ge(): Fatal error!' );
  end

  if ( nargin < 2 )
    ny = nx;
    fprintf ( 1, '\n' );
    fprintf ( 1, '  NY was not supplied.  Setting NY = NX = %d.\n', ny );
  end

  if ( nargin < 3 )
    n = wathen_order ( nx, ny );
    fprintf ( 1, '\n' );
    fprintf ( 1, '  N was not supplied.  N = %d\n', n );
  end

  a = zeros ( n, n );

  em = [
     6.0, -6.0,  2.0, -8.0,  3.0, -8.0,  2.0, -6.0;
    -6.0, 32.0, -6.0, 20.0, -8.0, 16.0, -8.0, 20.0;
     2.0, -6.0,  6.0, -6.0,  2.0, -8.0,  3.0, -8.0;
    -8.0, 20.0, -6.0, 32.0, -6.0, 20.0, -8.0, 16.0;
     3.0, -8.0,  2.0, -6.0,  6.0, -6.0,  2.0, -8.0;
    -8.0, 16.0, -8.0, 20.0, -6.0, 32.0, -6.0, 20.0;
     2.0, -8.0,  3.0, -8.0,  2.0, -6.0,  6.0, -6.0;
    -6.0, 20.0, -8.0, 16.0, -8.0, 20.0, -6.0, 32.0 ]';

  node = zeros(8,1);

  for j = 1 : ny

    for i = 1 : nx
%
%  For the element (I,J), determine the indices of the 8 nodes.
%
      node(1) = ( 3 * j     ) * nx + 2 * j + 2 * i + 1;
      node(2) = node(1) - 1;
      node(3) = node(1) - 2;

      node(4) = ( 3 * j - 1 ) * nx + 2 * j + i - 1;
      node(8) = node(4) + 1;

      node(5) = ( 3 * j - 3 ) * nx + 2 * j + 2 * i - 3;
      node(6) = node(5) + 1;
      node(7) = node(5) + 2;

      rho = 50.0 * rand ( 1, 1 );

      for krow = 1 : 8
        for kcol = 1 : 8
          a(node(krow),node(kcol)) = a(node(krow),node(kcol)) ...
            + rho * em(krow,kcol);
        end
      end

    end
  end

  return
end